In [ ]:
import subprocess
import tempfile

import os
import sys
sys.path.append(f'{os.environ["HOME"]}/Projects/planckClusters/catalogs')
from load_catalogs import load_PSZcatalog
                
# parallel processor
from utilities import parallel_process, check_exe, system_call
from utilities import get_immediate_subdirectories, get_immediate_subfiles

In [ ]:
# this allows us to run the pipeline not interactively. 
os.environ['HEADASNOQUERY'] = ''
os.environ['HEADASPROMPT'] = '/dev/null'

In [ ]:
def run_pipeline(name, outpath, overwrite=False):

    if not os.path.isdir(f'{outpath}/{name}'):
        return
    
    if not check_exe('xrtpipeline', verb=False):
        return
    
    # get a list of the XRT obs. 
    obs = get_immediate_subdirectories(f'{outpath}/{name}')
        
    # if there aren't any observations, keep going
    if not len(obs):
        return
    
    if not os.path.isdir(f'{outpath}/{name}/reduced'):
        os.makedirs(f'{outpath}/{name}/reduced')
    
    # get a list of the reduced obs.
    reduc = get_immediate_subdirectories(f'{outpath}/{name}/reduced')
    
    for ob_dir in obs:
        ob_id = ob_dir.split('/')[-1]
        reduc_ids = [r.split('/')[-1] for r in reduc]
        evts = False
        expm = False
        if ob_id == 'reduced':
            continue
        elif ob_id == 'spec_files':
            continue
        else:
            try:
                for f in get_immediate_subfiles(f'{outpath}/{name}/reduced/{ob_id}'):
                    if 'po_cl.evt' in f:
                        evts = True
                    elif 'po_ex.img' in f:
                        expm = True
                    else:
                        continue    
                if evts and expm and not overwrite:
                    continue
            except FileNotFoundError: # happens when nothing has been reduced
                pass
                    
        cmd = (f'xrtpipeline indir={ob_dir} outdir={outpath}/{name}/reduced/{ob_id} steminputs=sw{ob_id} '
                    'srcra=OBJECT srcdec=OBJECT datamode=PC cleanup=yes vigflag=yes clobber=yes ')
        
        stdout, stderr = system_call_env(cmd, shlexify=False)
        
        # log the output
        log_file = f'{outpath}/{name}/reduced/{ob_id}_reduce.log'
        with open(log_file, 'w') as f:
            f.writelines(stdout)

    return

In [ ]:
def validate():
    '''This has been rolled into the main function above''' 
    
    PSZs = get_immediate_subdirectories('./data_full')
    for psz in tqdm_notebook(PSZs):
        # get a list of the XRT obs. 
            for ob_dir in get_immediate_subdirectories(psz):
                # check for events and exposure map
                evts = False
                expm = False
                if ob_id == 'reduced':
                    continue
                else:
                    for f in get_immediate_subfiles(f'{psz}/reduced/{ob_id}'):
                        if 'po_cl.evt' in f:
                            evts = True
                        elif 'po_ex.img' in f:
                            expm = True
                        else:
                            continue
                
                    if not evts and expm:
                        print(f'{psz}/reduced/{ob_id} NOT VALID')

In [ ]:
'''
Running the xrtpipeline or really any ftool doesn't always work well when run in parallel. 

You should run everything once (or twice) with n_jobs=6 (or some number > 1) and then a final
time with n_jobs=1. This makes sure that the majority of the work gets done in parallel, and then
things are finallized serially. This notebook won't redo work, so if the data products are created
during the parallel processing, it won't be redone when run serially. 

Another issue is that some of the observations are in "window timing" or WT mode. The pipeline
script only looks at data in "photon counting" or PC mode. This causes a lot observations to be run
but no reduced data prodcuts will be produced. If you look at the logs, it will say that no events files
are found in the directory tree. If you look at the downloaded data, you will see a bunch of files with 
"wt" in them and no "pc" files.

This notebook considers those observations as "not done" so it tries to redo them. That means that every
time you run this notebook it will find work to do, even though those observations aren't any good to us.

'''


# get file data
data = load_PSZcatalog()
data = data.sort_values('NAME')

outpath = './data_full_new'

arr = [{'name':n.replace(' ', '_'), 'outpath':outpath, 'overwrite':False} for n in data['NAME']]
parallel_process(arr, run_pipeline, use_kwargs=True, n_jobs=1)


In [ ]:
# outpath = './data_full_new'
# name = 'PSZ2_G110.28-87.48'
# run_pipeline(name, outpath, False)